### Centaurus A
[TeVCat Gamma-Ray Source Summary](http://tevcat.uchicago.edu/?mode=1;id=122)</br>

[NASA/IPAC Extragalactic Database](https://ned.ipac.caltech.edu/byname?objname=Centaurus+A&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1)

### NGC 1275
http://tevcat.uchicago.edu/?mode=1;id=212</br>

https://ned.ipac.caltech.edu/byname?objname=NGC+1275&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1

In [ ]:
# from .core import 

import matplotlib.pyplot as plt # A collection of command style functions

import numpy as np

from astropy import units as u
from astropy.units import  Quantity
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord

from feupy.utils.string_handling import name_to_txt
from feupy.utils.units import Jy_to_erg_by_cm2_s
from feupy.catalog import CATALOG_REGISTRY
from feupy.analysis.config_cntr import CounterpartsConfig
from feupy.analysis.core_cntr import Counterparts

from feupy.plotters import generate_marker_set
from feupy.target import Target
from feupy.roi import ROI
from feupy.utils.table import write_tables_csv

from gammapy.utils.scripts import make_path

from gammapy.modeling.models import Models, LogParabolaSpectralModel, SkyModel
from gammapy.modeling import Fit

In [ ]:
from feupy.scripts.ipynb_to_gallery import convert_ipynb_to_gallery 

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt # A collection of command style functions

In [ ]:
from astropy import units as u
from astropy.units import  Quantity
from astropy.coordinates import SkyCoord
from astropy.table import Table

from gammapy.utils.scripts import make_path
from gammapy.datasets import Datasets, FluxPointsDataset

from gammapy.estimators import FluxPoints
from gammapy.modeling.models import Models, LogParabolaSpectralModel, SkyModel
from gammapy.modeling import Fit

from feupy.utils.string_handling import name_to_txt
from feupy.catalog import CATALOG_REGISTRY
from feupy.analysis.config_cntr import CounterpartsConfig
from feupy.analysis.core_cntr import Counterparts
from feupy.plotters import generate_marker_set
from feupy.target import Target
from feupy.roi import ROI

from core import get_data_VizieR_byname

In [ ]:
source_name = 'NGC 1275'
name_file = name_to_txt(source_name)
position = SkyCoord.from_name(source_name)

In [ ]:
outdir = f"./{name_file}"
outdir_path = make_path(outdir)
outdir_path.mkdir(parents=True, exist_ok=True)

datasets_path = make_path(f"{outdir_path}/datasets/")
datasets_path.mkdir(parents=True, exist_ok=True)

figures_path = make_path(f"{outdir_path}/figures/")
figures_path.mkdir(parents=True, exist_ok=True)

data_path = make_path(f"./data/{outdir}")
data_path.mkdir(parents=True, exist_ok=True)

In [ ]:
config = CounterpartsConfig()

config.general.outdir = outdir
config.general.datasets_file = f"{datasets_path}/datasets.yaml"
config.general.models_file = f"{datasets_path}/models.yaml"

In [ ]:
# catalog = CATALOG_REGISTRY.get_cls("gamma-cat")()
# source = catalog[source_name]
# print(source.info())

In [ ]:
file = f'{data_path}/table_photandseds.csv'
table = Table.read(file)
table

In [ ]:
file = f'{data_path}/table_qlphot.csv'
table = Table.read(file)
table

In [ ]:
columns = ['Spectral Region',
 'Band',
 'Apparent Mag or Flux',
 'Reference code',
 'Absolute Mag or νLν [W]',
 'νLν [L☉(Bolometric)]']

In [ ]:
table_ =  table[columns]
table_

In [ ]:
write_tables_csv(table_, path_file=data_path, file_name= 'table_qlphot_')

In [ ]:
file = f'{data_path}/table_qlphot_.csv'
table_ = Table.read(file)
table_

In [ ]:
!pip install pylatex

In [ ]:


import csv
def csv_to_latex(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        latex_table = "\\begin{tabular}{lcr}\n" + " & ".join(headers) + " \\\\\n\\hline\n"
        for row in reader:
            latex_table += " & ".join(row) + " \\\\\n"
        latex_table += "\\end{tabular}"
    return latex_table

table = 
print(csv_to_latex(f'{data_path}/table_qlphot.csv'))

In [ ]:
datasets_VizieR = get_data_VizieR_byname(table)

In [ ]:
datasets = Datasets(datasets_VizieR)

In [ ]:
xaxis_label = r'$\rm{E\ [TeV] }$'
xaxis_units = u.Unit('TeV')

yaxis_label = r'$\rm{E^2\ J(E)\ [erg\ cm^{-2}\ s^{-1}] }$'
yaxis_units = u.Unit('erg cm-2 s-1')

energy_bounds = [1e-22, 1e3] * u.TeV
ylim = [1e-16, 1e-8]

leg_place = dict(
    bbox_to_anchor = (1, 0.5), # Set legend outside plot
    ncol=1, 
#     loc='upper left', 
)
markers = generate_marker_set(datasets.names, 5)

In [ ]:
markers

In [ ]:
markers

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

kwargs = {"ax": ax, "sed_type": "e2dnde"}

for index, d in enumerate(datasets):
    marker=markers['marker'][index] 
    label=markers['label'][index] 
    markersize=markers['markersize'][index] 
    color=markers['color'][index] 

    d.data.plot(
        label=d.name, 
        c=color,
        marker=marker,
        markersize=markersize,
        ls='None',
        mec='k',
        mew=0.5,
        elinewidth=0.7,
        capsize=1.5,
        **kwargs
    )
    
ax.yaxis.set_units(yaxis_units)
ax.xaxis.set_units(xaxis_units)

ax.set_xlabel(xaxis_label)
ax.set_ylabel(yaxis_label)

ax.set_xlim(energy_bounds)
ax.set_ylim(ylim)

ax.legend(loc='upper left', ncol=1, fontsize=5, bbox_to_anchor=(1,1))

plt.title(f"{source_name}")

plt.savefig(f"{figures_path}/flux_points_VizieR_{name_file}")
plt.show()


In [ ]:
target = Target(
    name=source_name, 
    pos_ra=position.ra, 
    pos_dec=position.dec,
)

radius_roi = 0.25 * u.Unit("deg")

roi = ROI(target=target, radius=radius_roi)

config.roi = roi.dict
config.roi.catalogs = "all"

analysis = Counterparts(config)
analysis.run()


In [ ]:
analysis.df_sep

In [ ]:
# analysis.datasets.names

In [ ]:
# analysis.write_datasets()
# analysis.write_models()

In [ ]:
models = Models()
datasets = Datasets()

In [ ]:
# path_dir = datasets_dir
# datasets_HE = Datasets.read(filename=f'{path_dir}/datasets.yaml', filename_models=f'{path_dir}/models.yaml')

datasets_HE = analysis.datasets.copy()
models.extend(datasets_HE.models)
datasets.extend(datasets_HE.copy())

In [ ]:
xaxis_label = r'$\rm{E\ [MeV] }$'
xaxis_units = u.Unit('MeV')

yaxis_label = r'$\rm{E^2\ J(E)\ [erg\ cm^{-2}\ s^{-1}] }$'
yaxis_units = u.Unit('erg cm-2 s-1')

energy_bounds = [1e1, 1e7] * u.MeV
ylim = [1e-15, 1e-8]

leg_place = dict(
    bbox_to_anchor = (1, 0.5), # Set legend outside plot
    ncol=1, 
#     loc='upper left', 
)
markers = generate_marker_set(datasets.names, 5)

In [ ]:
spectral_model = datasets_HE.models[1].spectral_model

model_fit = SkyModel(
#     spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="model-fit",
)
models.append(model_fit)
display(model_fit.parameters.to_table())

In [ ]:
display(model_fit.parameters.to_table())

In [ ]:
datasets.models = model_fit
fitter = Fit()
result_fit = fitter.run(datasets=datasets)
display(model_fit.parameters.to_table())

In [ ]:
result_fit.success

In [ ]:
datasets.names

In [ ]:
energy_bounds = [1e1, 1e7] * u.MeV
ylim = [1e-15, 1e-8]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

kwargs = {"ax": ax, "sed_type": "e2dnde"}

for index, d in enumerate(datasets):
    marker=markers['marker'][index] 
    label=markers['label'][index] 
    markersize=markers['markersize'][index] 
    color=markers['color'][index] 

    d.data.plot(
        label=d.name, 
        c=color,
        marker=marker,
        markersize=markersize,
        ls='None',
        mec='k',
        mew=0.5,
        elinewidth=0.7,
        capsize=1.5,
        **kwargs
    )
    
model_fit.spectral_model.plot(energy_bounds=energy_bounds, label=model_fit.name, color="k", **kwargs)
model_fit.spectral_model.plot_error(energy_bounds=energy_bounds, **kwargs)
ax.yaxis.set_units(yaxis_units)
ax.xaxis.set_units(xaxis_units)

ax.set_xlabel(xaxis_label)
ax.set_ylabel(yaxis_label)

ax.set_xlim(energy_bounds)
ax.set_ylim(ylim)

ax.legend(loc='upper left', ncol=1, fontsize=5, bbox_to_anchor=(1,1))

plt.title(f"{source_name}")

plt.savefig(f"{figures_path}/SED_HE_{name_file}")
plt.show()


In [ ]:
analysis.datasets.models = models

In [ ]:
datasets.models = models

In [ ]:
datasets.extend(datasets_VizieR)
analysis.datasets = datasets

In [ ]:
display(models['model-fit'].parameters.to_table())

In [ ]:
display(datasets.models['model-fit'].parameters.to_table())

In [ ]:
datasets.models[0]

In [ ]:
datasets.write()

In [ ]:
config.write(path=f'{outdir}/config.yaml', overwrite=True)

In [ ]:
# config.read(config.general.config_file)

In [ ]:
datasets.write(filename=config.general.datasets_file, filename_models=config.general.models_file, overwrite=True)

In [ ]:
!jupyter nbconvert counterparts.ipynb  --to notebook --output 'counterparts_{source_name}' --output-dir '.'

In [ ]:
convert_ipynb_to_gallery('counterparts.ipynb', output_file_name=f'{outdir}/counterparts_{source_name}')